#prepare

In [1]:
%%capture
! pip install gensim

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine

#data

In [20]:
data = pd.read_csv('./Books.csv', encoding = 'utf-8')
data.drop_duplicates(['title'], inplace = True)
training_data = [TaggedDocument(words=data['overview'][i].split(), tags=[data['title'][i]]) for i in range(len(data))]

#train

In [4]:
# Train a doc2vec model
model = Doc2Vec(vector_size=100,
                epochs=40,
                dm=0,            # PV-DBOW / default 1
                dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
                window=10,        # distance between the predicted word and context words
                size=100,        # vector size
                alpha=0.025,     # learning-rate
                min_count=5,    # ignore with freq lower
                min_alpha=0.025, # min learning-rate
                workers=8,   # multi cpu
                hs = 1,          # hierar chical softmax / default 0
                negative = 10)

model.build_vocab(training_data)
model.train(training_data, total_examples=model.corpus_count, epochs=model.epochs)

#inference

In [5]:
# Infer a vector for new data
new_data = data['overview'][0]
new_vector = model.infer_vector(new_data.split())

# Calculate similarity scores for all pre-existing data
similarity_scores = [(index, 1 - cosine(new_vector, model.docvecs[index].reshape(1, -1))) for index in range(len(model.docvecs))]

k = 10
top_k = [(index,score) for index, score in sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:k]]

recommended_docs = [(data['title'][index],round(score,3))  for index, score in top_k]
recommended_docs

[('유도라 웰티의 소설작법', 0.461),
 ('번역의 일 - 번역이 없으면 세계도 없다', 0.452),
 ('우리말 맞춤법 띄어쓰기 - 모든 글쓰기의 시작과 완성', 0.44),
 ('사자소학 따라쓰기 - 손으로 쓰면서 마음에 새기는 인생 교과서', 0.399),
 ('마흔의 문장들', 0.395),
 ('자각의 연금술', 0.394),
 ('손잡지 않고 살아남은 생명은 없다 - 더불어 살아가기 위한 생명 이야기', 0.382),
 ('호모 데우스 - 미래의 역사', 0.382),
 ('사진으로 보는 우리 문화유산 ', 0.377),
 ('Magazine Makers - 독립 매거진을 창간해서 발행하는 5팀의 대화집', 0.377)]